In [9]:
#from models import keras_1, keras_2
from sklearn.model_selection import train_test_split
from preprocess import onehot,inv_onehot
from evaluation import *
from common import *
# train a feats model
from gen_feats_data import get_feats_data
from gen_fieldpos_data import get_fieldpos_data


In [10]:
feats_tr_data = get_feats_data(['2014','2015','2016'])
feats_val_data = get_feats_data(['2017'])

In [ ]:
print(feats_val_data)

In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import regularizers
import numpy as np

# class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
# sample_weights = compute_sample_weight('balanced', y_integers)

def keras_1(x_train,y_train,val_tuple):
#     from sklearn.utils import compute_class_weight
#     classWeight = compute_class_weight('balanced', np.unique(y_train), y_train)
#     classWeight = dict(enumerate(classWeight,1))
#     print(classWeight)
    
    from keras.optimizers import SGD
    opt = SGD(lr=0.05)
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    model = Sequential()
    model.add(Dense(100,input_dim = x_train.shape[1], activation = 'sigmoid'))
    model.add(Dense(200, activation = 'sigmoid',kernel_regularizer = regularizers.l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation = 'sigmoid'))
    model.add(Dense(3, activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer ='adam',metrics = ['accuracy'])

    model.fit(x_train,y_train,epochs = 50, batch_size = 32, validation_data = val_tuple)
    print(model.evaluate(val_tuple[0],val_tuple[1]))
    print(model.metrics_names)
    return model#.predict(val_tuple[0])
   
def keras_2(x_train,y_train,val_tuple):
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    from keras.optimizers import SGD
    opt = SGD(lr=0.05)
    """ Creates a Sequential model made out of Densely connected Neural Network layers. All parameters are configured as per specification for Q4"""

    model = Sequential()
    model.add(Dense(100,input_dim = x_train.shape[1], activation = 'sigmoid'))
    model.add(Dense(200, activation = 'sigmoid'))
    model.add(Dropout(0.3))
    model.add(Dense(3, activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer ='adam',metrics = ['accuracy'])

    model.fit(x_train,y_train,epochs = 50, batch_size = 32, validation_data = val_tuple)
    #print(model.evaluate())
    #print(model.metrics)
    return model#.predict(val_tuple[0])

In [ ]:
model = keras_1(feats_tr_data[0],onehot(feats_tr_data[1]),(feats_val_data[0],onehot(feats_val_data[1])))
# inv_pred = inv_onehot(feats_pred).reshape(1,-1)[0]
# acc = accuracy_score(inv_pred,feats_val_data[1])
# #acc


In [14]:
feats_test_data = get_feats_data(['2018'])

In [15]:
print(model)
test_pred = model.predict(feats_test_data[0])
print(test_pred)
inv_test_pred = inv_onehot(test_pred).reshape(1,-1)[0]


[[0.55901843 0.24686281 0.17961143]
 [0.1782027  0.2628504  0.62956333]
 [0.50141716 0.2510468  0.2212069 ]
 ...
 [0.49208578 0.25220537 0.2282843 ]
 [0.595212   0.24312663 0.15686604]
 [0.24575745 0.2617829  0.5106326 ]]


In [16]:
fieldpos_tr_data = get_fieldpos_data(['2014','2015','2016'])
fieldpos_val_data = get_fieldpos_data(['2017'])

In [17]:
fieldpos_test_data = get_fieldpos_data(['2018'])

In [18]:
model_2 = keras_2(fieldpos_tr_data[0],onehot(fieldpos_tr_data[1]),(fieldpos_val_data[0],onehot(fieldpos_val_data[1])))
# print(fieldpos_pred)
# inv_pred = inv_onehot(fieldpos_pred).reshape(1,-1)[0]
# print(inv_pred)

#accuracy_score(inv_pred,fieldpos_val_data[1])
print(model_2)
test_pred_2 = model_2.predict(fieldpos_test_data[0])
inv_test_pred_2 = inv_onehot(test_pred_2).reshape(1,-1)[0]


D:\Apps\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
D:\Apps\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEnco

Train on 1140 samples, validate on 380 samples
Epoch 1/50
1140/1140 [==============================] - 0s 382us/step - loss: 0.6289 - acc: 0.6690 - val_loss: 0.5815 - val_acc: 0.7184
Epoch 2/50
1140/1140 [==============================] - 0s 71us/step - loss: 0.6037 - acc: 0.6947 - val_loss: 0.5780 - val_acc: 0.7140
Epoch 3/50
1140/1140 [==============================] - 0s 65us/step - loss: 0.6064 - acc: 0.6962 - val_loss: 0.5789 - val_acc: 0.7114
Epoch 4/50
1140/1140 [==============================] - 0s 73us/step - loss: 0.5952 - acc: 0.6927 - val_loss: 0.5817 - val_acc: 0.7096
Epoch 5/50
1140/1140 [==============================] - 0s 65us/step - loss: 0.5931 - acc: 0.6924 - val_loss: 0.5821 - val_acc: 0.7070
Epoch 6/50
1140/1140 [==============================] - 0s 62us/step - loss: 0.5933 - acc: 0.7003 - val_loss: 0.5812 - val_acc: 0.7026
Epoch 7/50
1140/1140 [==============================] - 0s 66us/step - loss: 0.5879 - acc: 0.7000 - val_loss: 0.5826 - val_acc: 0.7070
Epoch 8

In [19]:



it = feats_val_data[0].iterrows()

for i in range(0,len(feats_val_data[1])):
    print(feats_pred[i], fieldpos_pred[i],feats_val_data[1][i],  [feats_pred[i][0] +   fieldpos_pred[i][0],  feats_pred[i][1] + fieldpos_pred[i][1],  feats_pred[i][2] + fieldpos_pred[i][2]])


NameError: name 'feats_pred' is not defined

In [ ]:
import operator

it = feats_val_data[0].iterrows()
z = 0
for i in range(0,len(feats_val_data[1])):
    a = [0,feats_pred[i][0],feats_pred[i][1],feats_pred[i][2],0,fieldpos_pred[i][0], fieldpos_pred[i][1],fieldpos_pred[i][2]]
    m = max(a)
    i = a.index(m)
    if (i)%4 ==feats_val_data[1][i] :
        z = z + 1
z/len(feats_val_data[1])

In [ ]:
def generate_final_table(fixtures, results):
    team_points = {}

    for i in range(10):
        team_points[fixtures.iloc[i]['HomeTeam']] = 0
        team_points[fixtures.iloc[i]['AwayTeam']] = 0

    for i in range(380):
        result = results[i]
        fixture = fixtures.iloc[i]
        if(result == 1):
            team_points[fixture['HomeTeam']] += 3
        elif(result == 2):
            team_points[fixture['HomeTeam']] += 1
            team_points[fixture['AwayTeam']] += 1
        elif(result == 3):
            team_points[fixture['AwayTeam']] += 3

    sorted_team_points = sorted(team_points.items(), key=lambda kv: kv[1])
    sorted_team_points.reverse()
    return dict(sorted_team_points)

In [ ]:
import pandas as pd

results = []
fixtures = get_fixtures('2018')


In [ ]:

final_table = pd.DataFrame.from_dict(generate_final_table(fixtures, inv_test_pred_2), orient = 'index')
final_table

In [ ]:
feats_pred = test_pred
fieldpos_pred = test_pred_2
times = 0
count = 0
home_wins = 0

for i in range(0,len(fieldpos_test_data[1])):
    #print(fieldpos_test_data[1][i],feats_pred[i], fieldpos_pred[i], [(feats_pred[i][0] +   fieldpos_pred[i][0])/2,  (feats_pred[i][1] + fieldpos_pred[i][1])/2,  (feats_pred[i][2] + fieldpos_pred[i][2])/2])
    a = [(feats_pred[i][0] *  fieldpos_pred[i][0]),  (feats_pred[i][1] * fieldpos_pred[i][1]),  (feats_pred[i][2] * fieldpos_pred[i][2])]
    #a = [feats_pred[i][0],  feats_pred[i][1],  feats_pred[i][2] ]
    #a = [fieldpos_pred[i][0],  fieldpos_pred[i][1],  fieldpos_pred[i][2] ]

    m = max(a)
    it = a.index(m) + 1
    #predict vs actual
    if it == fieldpos_test_data[1][i]:
        times = times + 1
        if it == 1 or it == 3:
            home_wins +=1
        
    count = count + 1
    if count == 131:
        break
print(times, count)
print(times/count)
print(home_wins/count)